#### Deep Learning
#### Autores: Mariana David 
#### Autores: Pablo Escobar 
#### Laboratorio 6 y 7

# Red neuronal basado en el filtro colaborativo


NNRM


In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.layers import Input, Embedding, Flatten, Concatenate, Dense
from keras.models import Model
from sklearn.preprocessing import LabelEncoder

# Configurar la TPU en Google Colab
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

with strategy.scope():
    # Carga los datos
    users_data = pd.read_csv('archive/Users.csv')
    books_data = pd.read_csv('archive/Books.csv')
    ratings_data = pd.read_csv('archive/Ratings.csv')

    users_data = users_data.sample(frac=0.1, random_state=1).reset_index(drop=True)
    books_data = books_data.sample(frac=0.1, random_state=1).reset_index(drop=True)
    ratings_data = ratings_data.sample(frac=0.1, random_state=1).reset_index(drop=True)

    ratings_data['ISBN'] = ratings_data['ISBN'].str.rstrip('X')

    label_encoder = LabelEncoder()
    ratings_data['ISBN'] = label_encoder.fit_transform(ratings_data['ISBN'])

    # Definir las incrustaciones de usuario y libro
    num_users = len(users_data) + 1
    num_books = len(books_data) + 1
    embedding_dim = 200

    user_input = Input(shape=[1], name="User-Input")
    user_embedding = Embedding(num_users, embedding_dim, name="User-Embedding")(user_input)
    user_vec = Flatten(name="Flatten-Users")(user_embedding)

    book_input = Input(shape=[1], name="Book-Input")
    book_embedding = Embedding(num_books, embedding_dim, name="Book-Embedding")(book_input)
    book_vec = Flatten(name="Flatten-Books")(book_embedding)

    # Concatenar las incrustaciones de usuario y libro
    concat = Concatenate()([user_vec, book_vec])

    # Capas densas para la predicción de calificación
    dense1 = Dense(128, activation='relu')(concat)
    dense2 = Dense(64, activation='relu')(dense1)
    output = Dense(1, activation='linear')(dense2)

    model = Model(inputs=[user_input, book_input], outputs=output)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

    history = model.fit([ratings_data['User-ID'], ratings_data['ISBN']], ratings_data['Book-Rating'], epochs=30, verbose=1)

# Hacer predicciones
user_id = 123  # ID de usuario deseado
book_id = 456  # ID de libro deseado
predicted_rating = model.predict([np.array([user_id]), np.array([book_id])])

print(f'Predicción de calificación para el usuario {user_id} y el libro {book_id}: {predicted_rating[0][0]}')

KeyError: 'COLAB_TPU_ADDR'